## no OCR version notebook
### with data from 2022 and 2023
at the end of this notebook, run the model you want to try out, run the seed and generate text.

this model has 50 epochs

In [1]:
# NOTE: this will download the model for the NLP you only need to do it once
try:
    import spacy
    nlp = spacy.load("de_dep_news_trf")
except:
    !python -m spacy download de_dep_news_trf

c:\Users\miche\Documents\GitHub\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\miche\Documents\GitHub\.venv\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\miche\Documents\GitHub\.venv\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\miche\Documents\GitHub\.venv\Lib\site-packages\thinc\shims\pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default p

In [2]:
# import spacy_transformers
# from spacy import displacy
# from spacy.lang.fr.examples import sentences

In [3]:
# please install requirements.txt first
import spacy
from spacy.tokens import Doc, DocBin, Token
from spacy.vocab import Vocab
import tf_keras as keras
from tf_keras.preprocessing.text import Tokenizer
import numpy as np
from tf_keras.models import Sequential
from tf_keras.layers import Dense, LSTM, Embedding
from random import randint
from tf_keras.models import load_model
from tf_keras.preprocessing.sequence import pad_sequences
from pickle import dump, load
import random
import os

In [4]:
BASE_DIR = os.getcwd()

prepareing a little bit more the data

In [5]:
# loading the extracted data
doc_bin = DocBin().from_disk(BASE_DIR + "/data/data_noOCR_22-23.spacy")
# needed for the vocab
nlp = spacy.load("de_dep_news_trf")

all_docs_sents = [i.sents for i in doc_bin.get_docs(nlp.vocab)]
all_sents = [i for j in all_docs_sents for i in j if len(i) >= 35]
tokens = [i.text for j in all_sents for i in j if not i.pos_ in ["X", "SPACE"]]
text = " ".join(tokens)
text = text.replace("- ", "")
tokens = text.split(" ")
filter_symbols = [')', '(', '/', '„', '“', '–', '%', '…', ':', ';', '�', '-', '[', ']', '≤', '=', '_', '<', '•', '+', '>', '∙', '°', '*', '−', '‘', '\uf036', '\uf02c', '·', '∑', '\uf06c', '\uf04f', '\uf028', '\uf026', '´', 'u', '×', '&', 'µ', '’', '≈', 'τ', '≥', '\uf03c', '\uf035', '□', '±', 'σ', 'ω', '”', '\uf027', '‚', '\uf023', '\uf037', 'ü', '∗', 'α', 'ª', '\uf040', '\uf03b', '√', 'γ', '\uf02f', '\uf033', 'δ', '«', '»', 'á', '\uf045', '𝑐', '∫', '\uf031', '\uf04b', '\uf030', '\uf039', '\uf029', '~', '→', '≪', '"', '®', '\xad', '\uf02b', '\uf02a']
tokens = [i for i in tokens if not i in filter_symbols]

c:\Users\miche\Documents\GitHub\.venv\Lib\site-packages\thinc\shims\pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, 

In [6]:
len(tokens)

422399

#50 words --> network predict #51
Here we will create a text sequence of 50 words, 51th will be the prediction word


In [7]:
train_len = 50 + 1
text_sequences = []

for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

#### loading pickle fo the tokenizer

In [8]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(text_sequences)
tokenizer = load(open(BASE_DIR + '/model/demo_tokenizer','rb'))

In [9]:
sequences = tokenizer.texts_to_sequences(text_sequences)

Getting the size of our vocabulary, to get the unique words across the whole document

In [10]:
vocabulary_size = len(tokenizer.word_counts)
vocabulary_size

35743

changing list into numpy array
where each row in the array represents a single textfragment, with lenght of 51

In [11]:
sequences = np.array(sequences)
sequences

array([[ 1276,     2,     4, ...,    13,   401,    26],
       [    2,     4, 11580, ...,   401,    26,    22],
       [    4, 11580,    10, ...,    26,    22,     9],
       ...,
       [   53,     4,    13, ...,    13, 35743,    99],
       [    4,    13, 17441, ..., 35743,    99,  1452],
       [   13, 17441,   176, ...,    99,  1452,   600]])

### Create the LSTM Based Model
Split the Data into Featureas and Labels

X will be the first n words of sequence

y will be the next word after the sequence

Fit the model

In [12]:
X = sequences[:,:-1]

In [13]:
y = sequences[:,-1]
y.shape

(422348,)

In [14]:
seq_len = X.shape[1] #setting the seq_len to 50

In [15]:
X.shape #ammount of sequences, each containing 50 words

(422348, 50)

### Training the model
LSTM layer to deal with the sequences, and Embedding layer to deal with the vocabulary

Note: at the LSTM the defining number of neurons should be a multipe of the seq_len, tourned out a 2 worked out well and made calculations quite fast. took only over night...

This part is a "left over" of the training to show the model used.

In [16]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size,seq_len,input_length=seq_len))
    model.add(LSTM(seq_len*2, return_sequences = True))
    model.add(LSTM(seq_len*2))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(vocabulary_size,activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [17]:
model = create_model(vocabulary_size+1,seq_len)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            1787200   
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 35744)             3610144   
                                                                 
Total params: 5548244 (21.16 MB)
Trainable params: 5548244 (21.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Generating new Text based on Seed Input

In [18]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    this is the frist try of the generate text, without beam_search
    it also gives the needed data for the generate_text_2 function
    differnt possible returns!
    
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model, special case of num_gen_words == 1
    '''

    # Final Output
    output_text = []

    # Intial Seed Sequence
    input_text = seed_text

    # Create num_gen_words
    for i in range(num_gen_words):

        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]

        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

        # Predict Class Probabilities for each word
        pred_word_ind = model.predict(pad_encoded, verbose=0)[0]

        # Grab word
        pred_word = tokenizer.index_word[np.argmax(pred_word_ind)]

        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word

        output_text.append(pred_word)
        
		# this part is for the generate_text_2 to return more data like probabilites
        if num_gen_words == 1:
            best_3_w = np.argsort(pred_word_ind)[-1:-4:-1]
            return best_3_w, pred_word_ind[best_3_w], pred_word_ind


    # Make it look like a sentence.
    return ' '.join(output_text)

In [19]:
def generate_text_2(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    This is the good one, with beam_search
    
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []

    # Intial Seed Sequence
    input_text = seed_text
    
    # Little Beam-search-like
    beam_search = []
    
    # Create num_gen_words
    for i in range(num_gen_words):

        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]

        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

        # Predict Class Probabilities for each word
        pred_word_ind = model.predict(pad_encoded, verbose=0)[0]
        
        # Testing out grab best 3 words
        best_3_w = np.argsort(pred_word_ind)[-1:-4:-1]
        beam_search.append(best_3_w)
        tokens_array = []
        tokens_prob_array = []
        for i in best_3_w:
            # Grab word
            pred_word = tokenizer.index_word[i]
            
            # Update the sequence of input text (shifting one over with the new word)
            try_text = input_text + ' ' + pred_word
            tokens, token_prop, _ = generate_text(model=model, tokenizer=tokenizer, seq_len=seq_len, seed_text=try_text, num_gen_words=1)
            tokens_array.append(tokens)
            tokens_prob_array.append(token_prop * pred_word_ind[i])
        id_token = np.argmax(np.array(tokens_prob_array).reshape(9,))
        ind_1st_word = id_token.__floordiv__(3)

        ind_2nd_word = id_token - (3 * ind_1st_word)
        counter = 0
        while best_3_w[ind_1st_word] == tokens_array[ind_1st_word][ind_2nd_word]:
            ind_1st_word = np.random.randint(0,3)
            counter += 1
            if counter > 4:
                ind_2nd_word = np.random.randint(0,3) # if all fails, random pick, depending on the model, it should not happen
        first_word = tokenizer.index_word[best_3_w[ind_1st_word]]
        second_word = tokenizer.index_word[tokens_array[ind_1st_word][ind_2nd_word]]
        
        input_text += ' ' + first_word #+ " " + second_word    # as in the readme.md this didn't work out well

        output_text.append(first_word)
        # output_text.append(second_word)

    # Make it look like a sentence.
    return ' '.join(output_text)

### loading models
Here we are loading unnesseary all models, its for convinience, so you can just run the block of the model's epoch you want to try out.

In [20]:
# 5
model = keras.saving.load_model(BASE_DIR + '/model/alfa_dl_model05-0.14.h5')

In [21]:
# 21
model = keras.saving.load_model(BASE_DIR + '/model/alfa_dl_model10-0.21.h5')

In [22]:
# 31
model = keras.saving.load_model(BASE_DIR + '/model/alfa_dl_model31-0.45.h5')

In [23]:
# 41
model = keras.saving.load_model(BASE_DIR + '/model/alfa_dl_model41-0.52.h5')

In [24]:
# 50
model = keras.saving.load_model(BASE_DIR + '/model/alfa_dl_model50-0.55.h5')

# Run all blocks below again and enjoy the results
the generate_text without beam_search most times fails even at 25 only.

In [25]:

#random.seed(112) 
ramdom_pick = random.randint(0,len(text_sequences))
random_seed_text = text_sequences[ramdom_pick]
seed_text = ' '.join(random_seed_text)
seed_text

'z. B. durch andere Personen , die bei der Messung zugegen sind oder die Messperson während der Ablesung des Messergebnisses , ca. 27 Feldstärkemes sung , ohne Wiederholbarkeit ca. 40 , einschließlich Wiederholbarkeit treten unregelmäßig auf und resultieren in unvorhersehbaren Schwankungen in Größe und Vorzeichen , entstehen durch nicht zu beeinflussende'

if you now go back up and only run 1 block of the models, you can try out the different epochs, and see how it evolved.

In [26]:
generate_text_2(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=50)

'unsystematische änderungen der messbedingungen z. untergruppen oder befähigungsnachweis zum elektrofahrzeug v emkg der gleichung rundfunk den messebenen 1 , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,'

In [27]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

'unsystematische änderungen der messbedingungen z. untergruppen , , , , , , , , , , , , , , , , , , ,'

In [28]:
# generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=1) # used for looking up some numbbers
# 1st array are the token, 2nd their probabilities, 3rd all probabilities of all tokens in order of the vocab_size  

In [29]:
# used for some testing, to figure out how certain the AI is when certain hickups appeared
# NOTE: there is a import down here

# _, __, array = generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=1)

# import pandas as pd
# df = pd.DataFrame(array)
# df.describe()